In [173]:
import blocksci
import matplotlib.pyplot as plt
import matplotlib.ticker
import collections
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
%matplotlib notebook

# parser_data_directory should be set to the data-directory which the blocksci_parser output
chain = blocksci.Blockchain("/blocksci/bitcoin.json")

In [174]:
# of inputs in aggregation : 18 – 25
# only one output (aggregated to)
# Amount of bitcoin in the big wallet: 15 to 100 (majority)
# Time between input transaction and the aggregation transaction: 1 hour
# Time between aggregation transaction and it being spent: 1 hour

def is_BTC_e(tx):
    # ** tx is a TxProxy type, so all of its fields are also proxy, like 
    # tx.input_count is an intProxy that cannot be compared to an int like
    # 18. **
    
    if (tx.input_count < 18 or tx.input_count > 25): 
        print("Input_count out of range, so false")
        return False
    
#     print('2')
#     print(tx.output_count)
    if (tx.output_count != 1):
        return False
    
#     print('3')
#     print(tx.output_value/1e8)
    val = tx.output_value/1e8
    if (val < 15 or val > 100):
        return False
    
#     print('4')
    output = tx.outputs[0]
#     print(output)
    if (not output.is_spent):
        return False
    
#     print('5')
    # spent within an hour
    time = tx.block_time
    delta = output.spending_tx.block_time - time
#     print(delta)
    if (delta > timedelta(minutes=90)):
        return False 
    
    return True
    
    # aggregated addresses were received within an hour
    
#     for addr in tx.inputs:
#         if (addr.block_t)


In [175]:
# testing on known transactions 
true = chain.tx_with_hash("c3377cc8f0f6002d5a1bd5e58e7676b495909a789e382577f5e9e09c364d1c89")
false = chain.tx_with_hash("2b20ae522e360c64b61a852bb0294ed7a16b5623a47c11919921917546ac0734")

In [176]:
print(is_BTC_e(true))
print("...")
# cast 'true to a txProxy, which is what happens when we run where - so if this works 
# correctly, then where will work.
print(is_BTC_e(blocksci.proxy.TxProxy(true)))

True
...
Input_count out of range, so false
False


In [177]:
print(is_BTC_e(blocksci.proxy.TxProxy(false)))
print(false.index)

Input_count out of range, so false
False
339783916


In [178]:
# this is my comparison function that returns a boolProxy, because it's used in 'where'
def select_BTC_e(tx):
    return blocksci.proxy.boolProxy(is_BTC_e(tx)) & ~tx.is_coinbase
#     return is_BTC_e(tx) & ~tx.is_coinbase

In [179]:
# test, ignore this
matches = chain[338950:338960].txes.where(lambda x: x.hash == tx.hash).to_list()
print(len(matches))

0


In [180]:
# btc_e_matches = chain[-557101:].txes.where(select_BTC_e).to_list()
# this should return > 1 matches, but currenctly returns 0 since select_BTC_e doesn't wokr
# where gives transactions using TxProxy to select_BTC_e
btc_e_matches = chain[338957:338960].txes.where(select_BTC_e).to_list()

print(len(btc_e_matches))
print(len(chain[338957:338960].txes.to_list()))

Input_count out of range, so false
0
1625


In [ ]:
# the range of blocks where 1Fs had transactions
btc_e_matches = chain[311793:478781].txes.where(select_BTC_e).to_list()

print(len(btc_e_matches))
